# StopDetection Test with Multi-Input Tools

# V1

把一整个任务都包好了，只是让GPT调了跑
好像没问题

In [7]:
import os
import pandas as pd
import skmob
from skmob import TrajDataFrame
from skmob.preprocessing import filtering
from skmob.preprocessing import detection

os.environ["LANGCHAIN_TRACING"] = "true"

from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0)
from langchain.tools import StructuredTool


def stop_detection(file_path:str) -> skmob.TrajDataFrame:
    """use the input file to do the stop detection task"""
    # Read the PLT file into a pandas DataFrame
    df = pd.read_csv(file_path, skiprows=6, header=None,
                     names=['lat', 'lon', 'zero', 'alt', 'date', 'date_str', 'time_str'])
    # Drop useless columns
    df = df.drop(columns=['zero', 'alt', 'date', 'date_str'])
    # Create a TrajectoryDataFrame from the DataFrame
    tdf = TrajDataFrame(df, latitude='lat', longitude='lon', datetime='time_str')
    # Preprocess the trajectory data by filtering out noise
    tdf = filtering.filter(tdf, max_speed_kmh=100)
    # Detect stops in the trajectory data
    stdf = detection.stay_locations(tdf, stop_radius_factor=0.5, minutes_for_a_stop=20.0, spatial_radius_km=0.2,
                                    leaving_time=True)
    # Return the stops as a pandas DataFrame
    return stdf


tool = StructuredTool.from_function(stop_detection)
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

task = "a stop detection task on data Geolife Trajectories 1.3. Find some information from the output, like how many rows are in the result, etc. File_path = '/Users/user/Downloads/scikitmob/20081023025304.plt'"
prompt = f"""
You are an AI spatial-temporal data assistant and want to make use of a python library called Scikit-mobility.
You need to write some python code to do a task.
The task is {task}."""


agent_executor.run(prompt)

Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15bb5e2b0>: Failed to establish a new connection: [Errno 61] Connection refused'))




> Entering new AgentExecutor chain...

Action:
```
{
  "action": "stop_detection",
  "action_input": {
    "file_path": "https://snap.stanford.edu/data/loc-brightkite_totalCheckins.txt.gz"
  }
}
```



Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2d1539ee0>: Failed to establish a new connection: [Errno 61] Connection refused'))


ValueError: could not convert string to float: '0\t2010-10-13T20:05:43Z\t39.7513\t-105.000121\td268093afe06bd7d37d91c4d436e0c40d217b20a'

# V2

把任务拆小一点，涉及pandas df传参
有点寄

In [10]:
import os
import pandas as pd
import skmob
from skmob import TrajDataFrame
from skmob.preprocessing import filtering
from skmob.preprocessing import detection

os.environ["LANGCHAIN_TRACING"] = "true"

from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0)
from langchain.tools import StructuredTool


def read_csv_from_file(file_path:str) -> pd.DataFrame:
    """Read the PLT file into a pandas DataFrame and do some basic data cleaning"""
    df = pd.read_csv(file_path, skiprows=6, header=None,
                     names=['lat', 'lon', 'zero', 'alt', 'date', 'date_str', 'time_str'])
    # Drop useless columns
    df = df.drop(columns=['zero', 'alt', 'date', 'date_str'])
    return df
    
def create_tdf(df:pd.DataFrame) ->skmob.TrajDataFrame:
    """Create a TrajectoryDataFrame from the pandas DataFrame and do some basic data cleaning"""
    tdf = TrajDataFrame(df, latitude='lat', longitude='lon', datetime='time_str')
    # Preprocess the trajectory data by filtering out noise
    tdf = filtering.filter(tdf, max_speed_kmh=100)
    return tdf

def stop_detection(tdf:skmob.TrajDataFrame) -> skmob.TrajDataFrame:
    """Detect stops in the trajectory data"""
    stdf = detection.stay_locations(tdf, stop_radius_factor=0.5, minutes_for_a_stop=20.0, spatial_radius_km=0.2,
                                    leaving_time=True)
    # Return the stops as a pandas DataFrame
    return stdf


tool = [StructuredTool.from_function(stop_detection),StructuredTool.from_function(read_csv_from_file),
        StructuredTool.from_function(create_tdf)]

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    tool,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

task = "a stop detection task on data Geolife Trajectories 1.3. Find some information from the output, like how many rows are in the result, etc. File_path = '/Users/user/Downloads/scikitmob/20081023025304.plt'"
prompt = f"""
You are an AI spatial-temporal data assistant and want to make use of a python library called Scikit-mobility.
You need to write some python code to do a task.
The task is {task}."""


agent_executor.run(prompt)

ValueError: Value not declarable with JSON Schema, field: name='tdf' type=TrajDataFrame required=True